In [45]:
import pandas as pd
import numpy as np

In [20]:
df2= pd.read_csv('/Users/dguiraldes/Documents/ISCI/intro_smart_meters/SML-dataset/halfhourly_dataset/halfhourly_dataset/block_0.csv')

In [21]:
data=df2[(df2['LCLid']=='MAC000002') & (df2['tstp']>='2013-01-01') & (df2['tstp']<'2014-01-01')].reset_index()[['tstp','energy(kWh/hh)']]

In [36]:
data['energy(kWh/hh)']=data['energy(kWh/hh)'].astype('float')

In [37]:
data['tstp']=pd.to_datetime(data['tstp'])

In [39]:
data['date']=data['tstp'].dt.date.apply(lambda dt: dt.replace(day=1))

In [42]:
monthly_data=data.groupby(['date']).sum().reset_index()

In [44]:
monthly_data['energy(kWh/hh)']

0     359.872000
1     381.622000
2     479.684000
3     362.588001
4     303.894000
5     299.931000
6     271.151000
7     269.891000
8     321.512000
9     378.171000
10    409.862000
11    376.470000
Name: energy(kWh/hh), dtype: float64

In [49]:
params={
    'cfm': np.ones(12)*684.730,
    'csp': np.ones(12)*0.508,
    'tr':np.ones(12)*12.784,
    'en':np.ones(12)*78.911,
    'pw':np.ones(12)*15.676,
    'bpw':np.ones(12)*18.326, 
    'pw_win':np.ones(12)*31.353,
    'bpw_win':np.ones(12)*36.583
}

In [68]:
out_df=monthly_data.copy()
for col in ['csp', 'tr', 'en', 'pw', 'bpw']:
    out_df[col]=out_df['energy(kWh/hh)']*params[col]
out_df['cfm']=params['cfm']


In [69]:
out_df['Total']=out_df[['cfm', 'csp', 'tr', 'en', 'pw', 'bpw']].sum(axis=1)

In [71]:
def winter_split(energy,limit):
    winter_energy=max(0,energy-limit)
    normal_energy= energy if winter_energy==0 else limit

In [70]:
out_df

,date,energy(kWh/hh),csp,tr,en,pw,bpw,cfm,Total
0,2013-01-01,359.872000,182.814976,4600.603649,28397.859400,5641.353474,6595.014274,684.73,46102.375773
1,2013-02-01,381.622000,193.863976,4878.655652,30114.173666,5982.306477,6993.604777,684.73,48847.334548
2,2013-03-01,479.684000,243.679472,6132.280255,37852.344116,7519.526382,8790.688982,684.73,61223.249207
3,2013-04-01,362.588001,184.194704,4635.324998,28612.181707,5683.929496,6644.787697,684.73,46445.148603
4,2013-05-01,303.894000,154.378152,3884.980897,23980.579442,4763.842346,5569.161446,684.73,39037.672283
5,2013-06-01,299.931000,152.364948,3834.317904,23667.855141,4701.718356,5496.535506,684.73,38537.521855
6,2013-07-01,271.151000,137.744708,3466.394383,21396.796553,4250.563074,4969.113224,684.73,34905.341942
7,2013-08-01,269.891000,137.104628,3450.286543,21297.368693,4230.811314,4946.022464,684.73,34746.323642
8,2013-09-01,321.512000,163.328096,4110.209409,25370.833440,5040.022114,5892.028914,684.73,41261.151973
9,2013-10-01,378.171000,192.110868,4834.538060,29841.851757,5928.208591,6930.361741,684.73,48411.801017
